In [1]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
print(torch.version)
print(torch.cuda.is_available())
from transformers.utils import is_torch_available, is_tf_available

print("Torch disponible:", is_torch_available())
print("TensorFlow disponible:", is_tf_available())

c:\Users\migue\Documents\GitHub\Practica_BAIN_4\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'torch.version' from 'c:\\Users\\migue\\Documents\\GitHub\\Practica_BAIN_4\\.env\\Lib\\site-packages\\torch\\version.py'>
True
Torch disponible: True
TensorFlow disponible: False


In [2]:
df = pd.read_csv('INPUT/emociones.csv')
df = df[['opinion', 'label']]
df = df.dropna()
df = df.sample(30, random_state=42).reset_index(drop=True)
print(df)

                       opinion     label
0          Es un gran servicio  positiva
1                   Es regular   neutral
2          Es un producto útil  positiva
3                 Es un fraude  negativa
4               Me gusta mucho  positiva
5               Es un desastre  negativa
6           Es un mal servicio  negativa
7        Es un producto inútil  negativa
8           No lo recomendaría  negativa
9       Me encantó el producto  positiva
10                 No me gusta  negativa
11               Es fantástico  positiva
12                  Me encanta  positiva
13         Es un buen producto  positiva
14                Es muy bueno  positiva
15          Es un mal servicio  negativa
16    Es una pérdida de dinero  negativa
17         No es nada especial   neutral
18       Es un producto normal   neutral
19                     Te odio  negativa
20         Es un buen servicio  positiva
21         Es un gran producto  positiva
22      Es un producto decente   neutral
23  Es un produc

In [3]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto" #comentario prueba
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [4]:
def predict_qwen(prompt):
    
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [5]:

# Lista para almacenar las predicciones
predictions_base = []
predictions_zero_shot = []
predictions_few_shot = []
predictions_cot = []

# Clasificar cada opinión y mostrar resultado
for row in df.itertuples():
    opinion = row.opinion
    label = row.label

    prompt_base = f"Clasifica la siguiente opinión en base a como interpretes la actitud del mensaje, dicha actitud puede ser vista como positiva, negativa o neutral: {opinion}"

    zero_shot_prompt = f"Clasifica la siguiente opinión como positiva, negativa o neutral: {opinion}"

    few_shot_prompt = f"""
    Aquí tienes algunas opiniones clasificadas:

    Opinión: 'Me encanta este producto. Es increíble.' → Sentimiento: positivo
    Opinión: 'Este servicio es terrible, nunca lo recomendaría.' → Sentimiento: negativo
    Opinión: 'El artículo está bien, pero no es excepcional.' → Sentimiento: neutral

    Clasifica esta opinión: {opinion}
    """

    cot_prompt = f"""
    Pensando en la siguiente opinión:

    Opinión: '{opinion}'

    El sentimiento puede ser positivo, negativo o neutral. ¿Qué opinas?
    """

    prompts = {
        "prompt": prompt_base,
        "zero-shot": zero_shot_prompt,
        "few-shot": few_shot_prompt,
        "chain-of-thought": cot_prompt
    }
    # Generar predicción con Zero-shot
    prompt_zero_shot = zero_shot_prompt.format(opinion)
    sentiment_zero_shot = predict_qwen(prompt_zero_shot)
    
    # Generar predicción con Few-shot
    prompt_few_shot = few_shot_prompt.format(opinion)
    sentiment_few_shot = predict_qwen(prompt_few_shot)

    # Generar predicción con el prompt base
    prompt = prompt_base.format(opinion)
    sentiment_base = predict_qwen(prompt)
    
    # Generar predicción con Chain-of-thought
    prompt_cot = cot_prompt.format(opinion)
    sentiment_cot = predict_qwen(prompt_cot)
    
    # Asignar sentimientos de acuerdo al texto generado
    def assign_sentiment(response):
        normalized_response = response.lower().strip()

        palabras_positivas = ["positivo", "positiva", "positive"]
        palabras_negativas = ["negativo", "negativa", "negative"]
        palabras_neutras = ["neutral", "neutro", "neutra"]

        if any(word in normalized_response for word in palabras_positivas):
            return "positiva"
        elif any(word in normalized_response for word in palabras_negativas):
            return "negativa"
        elif any(word in normalized_response for word in palabras_neutras):
            return "neutral"
        else:
            return "desconocida"
    
    # Asignar sentimientos a cada predicción
    predictions_base.append(assign_sentiment(sentiment_base))
    predictions_zero_shot.append(assign_sentiment(sentiment_zero_shot))
    predictions_few_shot.append(assign_sentiment(sentiment_few_shot))
    predictions_cot.append(assign_sentiment(sentiment_cot))

    # Imprimir resultados
    print(f"Opinión: {opinion} - Label: {label}\n → Zero-shot Sentimiento: {predictions_zero_shot[-1]}")
    print(f" → Few-shot Sentimiento: {predictions_few_shot[-1]}")
    print(f" → Chain-of-thought Sentimiento: {predictions_cot[-1]}")
    print(f" → Base Sentimiento: Sentimiento: {predictions_base[-1]}\n")

# Añadir las predicciones al DataFrame
df['prompt_base'] = prompt_base
df['prediccion_zero_shot'] = predictions_zero_shot
df['prediccion_few_shot'] = predictions_few_shot
df['prediccion_cot'] = predictions_cot

# Mostrar el DataFrame con los resultados
print(df[['opinion', 'label', 'prompt_base' ,'prediccion_zero_shot', 'prediccion_few_shot', 'prediccion_cot']])



Opinión: Es un gran servicio - Label: positiva
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: positiva
 → Chain-of-thought Sentimiento: positiva
 → Base Sentimiento: Sentimiento: positiva

Opinión: Es regular - Label: neutral
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: positiva
 → Chain-of-thought Sentimiento: positiva
 → Base Sentimiento: Sentimiento: positiva

Opinión: Es un producto útil - Label: positiva
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: positiva
 → Chain-of-thought Sentimiento: positiva
 → Base Sentimiento: Sentimiento: positiva

Opinión: Es un fraude - Label: negativa
 → Zero-shot Sentimiento: negativa
 → Few-shot Sentimiento: negativa
 → Chain-of-thought Sentimiento: positiva
 → Base Sentimiento: Sentimiento: negativa

Opinión: Me gusta mucho - Label: positiva
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: positiva
 → Chain-of-thought Sentimiento: positiva
 → Base Sentimiento: Sentimiento: positiva

Opinión: Es 

In [6]:
aciertos_df = pd.DataFrame({
    'prompt_base': predictions_base,
    'zero_shot': predictions_zero_shot,
    'few_shot': predictions_few_shot,
    'chain_of_thought': predictions_cot
})

aciertos = {
    'prompt_base': (aciertos_df['prompt_base'] == df['label']).sum(),
    'zero_shot': (aciertos_df['zero_shot'] == df['label']).sum(),
    'few_shot': (aciertos_df['few_shot'] == df['label']).sum(),
    'chain_of_thought': (aciertos_df['chain_of_thought'] == df['label']).sum()
}

print(f"Aciertos con la prompt base: {aciertos['prompt_base']} - Ratio: {aciertos['prompt_base'] / len(aciertos_df):.2f}")
print(f"Aciertos con la prompt zero shot: {aciertos['zero_shot']} - Ratio: {aciertos['zero_shot'] / len(aciertos_df):.2f}")
print(f"Aciertos con la prompt few shots: {aciertos['few_shot']}- Ratio: {aciertos['few_shot'] / len(aciertos_df):.2f}")
print(f"Aciertos con la prompt chain of thought: {aciertos['chain_of_thought']} - Ratio: {aciertos['chain_of_thought'] / len(aciertos_df):.2f}")

Aciertos con la prompt base: 26 - Ratio: 0.87
Aciertos con la prompt zero shot: 28 - Ratio: 0.93
Aciertos con la prompt few shots: 26- Ratio: 0.87
Aciertos con la prompt chain of thought: 14 - Ratio: 0.47
